In [1]:
using Revise
using LinearAlgebra
using AdaptiveTransportMap
using Statistics
using Distributions
using Test
using ForwardDiff
using SpecialFunctions
using OrthoMatchingPursuit

In [2]:
A = randn(10, 5)

10×5 Array{Float64,2}:
  0.846711   -2.05991    -0.671031    0.193168    0.767641
 -0.248853   -0.72072    -2.16088    -0.0822937   2.25991
  0.35429    -0.0769643  -0.116956   -1.1324      0.630609
 -1.24244     1.14009     1.62245     0.246204    0.570682
 -0.0320655   0.463823    0.114834   -0.664412   -0.01783
 -2.03965     0.507124    0.583453    0.625594   -0.286829
 -1.3586     -0.0640034   1.2489      0.757232    0.558471
  1.58707    -0.0418703  -0.369809    0.804867   -0.843747
  0.953096   -1.09407    -0.0136953  -0.162962    1.6028
  0.236479    0.844709   -0.779797    1.13624    -0.0156478

In [3]:
mean(A; dims = 1)[1,:]

5-element Array{Float64,1}:
 -0.09439655807283336
 -0.11017911270588691
 -0.054253711787882485
  0.17212404015337582
  0.52260662560226

In [11]:
# @testset "Test optimization for Nx=1 and p=3" begin
    Nx = 1
    Ne = 10
    p = 3
    γ = 2.0

    X = reshape([-1.5438
      -1.5518
       0.8671
      -0.1454
      -0.3862
       1.3162
      -0.7965
       0.1354
       0.4178
       0.8199],1,Ne)


    S = RadialMap(Nx, p; γ = γ)
    W = create_weights(S, X)
    center_std!(S, X)
    compute_weights(S, X, W)

    # With λ = δ = 0.0
    λ = 0.0
    δ = 0.0
    xopt = optimize(S.C[1], W, λ, δ);
    @show xopt

    @test norm(xopt - [0.453363732161572; 2.304194295945015; 0.0; 0.0; 0.0;2.552281803302206])<1e-10


    # With λ = 0.1 and δ = 1e-8
    λ = 0.1
    δ = 1e-8
    xopt = optimize(S.C[1], W, λ, δ);

    @test norm(xopt - [0.450882345511096; 2.295315495868833; 0.000000043115815; 0.000000032750391; 0.000000035316480; 2.547661799360081])<1e-10

# end

no = 0
xopt = [0.4533637321615722, 2.3041942959450146, 0.0, 0.0, 0.0, 2.552281803302206]
no = 0


Test Passed

In [34]:
Nx = 5
Ne = 100

X = randn(Nx, Ne) .* randn(Nx, Ne)

λ = 0.0
δ = 1e-8
γ = 2.0

2.0

In [3]:
@time Cgreedy = greedyfit(Nx, 2, X, 3, λ, δ, γ)

size(F) = (3, 100)


LoadError: [91mDimensionMismatch("for a right-sided multiplication, the second dimension of C, 4, must equal the second dimension of A, 3")[39m

In [23]:
# using BenchmarkTools
N = 200
nx = 597
λ = 0.0

ψ = rand(nx,N)
ψ_aug = zeros(N+nx, nx)

ψ_aug[1:N,:] .= ψ'
ψ_aug[N+1:N+nx,:] .= Matrix(sqrt(λ)*I, nx, nx)

F = qr(ψ_aug)

Q1 = zeros(N, nx)
Q1 .= view(F.Q*Matrix(I, N+nx, N+nx), 1:N, 1:nx)
Q1 .= (F.Q*Matrix(I,N+nx, N+nx))[1:N, 1:nx]
Q1 .= Matrix{Float64}(F.Q)[1:N, 1:nx];


ψbis  = rand(4, N) 

A = ψbis - (ψbis*Q1)*Q1';

In [24]:
F = qr(ψ')
size(F)

(200, 597)

In [21]:
ψbis

4×200 Array{Float64,2}:
 0.629605   0.326684  0.874646   0.0411056  …  0.363469  0.317491  0.934818
 0.22399    0.283091  0.53219    0.4247        0.527785  0.164593  0.871868
 0.566864   0.511157  0.700122   0.348717      0.723372  0.334182  0.952003
 0.0505716  0.995667  0.0391326  0.143356      0.769447  0.401063  0.300964

In [33]:
Matrix(qr(ψ_aug).Q)[1:200, 1:200]-Matrix(F.Q)

200×200 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …   1.94289e-16   7.04298e-16
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     -6.80012e-16  -2.498e-16
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      1.66533e-16  -1.39472e-15
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     -5.44703e-16  -4.28477e-16
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      4.60569e-16  -5.55112e-17
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …   3.46945e-17  -4.15466e-16
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     -3.48679e-16  -3.46945e-17
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      4.19803e-16   6.93889e-18
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     -7.21645e-16   1.08247e-15
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0      3.67761e-16   1.38778e-16
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …   4.0766e-16    1.52656e-16
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     -9.36751e-16  -1.11022e-16
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     -1.17961e-16   

In [30]:
Matrix(F.Q)

200×200 Array{Float64,2}:
 -0.0484646     0.00365827   0.0430735   …  -0.149698     0.023572
 -0.112344     -0.0339007    0.12418         0.0304372   -0.0260589
 -0.0447021     0.0304689   -0.0561091       0.171499     0.0426105
 -0.00403244   -0.0666002   -0.102289        0.0212374   -0.00875819
 -0.0509554     0.0016974    0.0248697      -0.00401667  -0.0389847
 -0.016677     -0.0124505   -0.0953902   …  -0.0280913   -0.00500914
 -0.0632723    -0.0343341    0.0449113       4.77836e-5   0.00135624
 -0.0183401     0.019794    -0.0860882      -0.0133249    0.0187791
 -0.0889232     0.0664824   -0.086204       -0.102532    -0.100871
 -0.019835     -0.0276584    0.0344452      -0.0465461    0.107352
 -0.000194863  -0.147459     0.0629499   …   0.0112554   -0.0544895
 -0.128356      0.0768003   -0.0981032       0.0149366   -0.112761
 -0.0391227    -0.10365      0.0158012      -0.041985    -0.0089801
  ⋮                                      ⋱               
 -0.0154384    -0.0936191    0.04

In [10]:
Q1 = zeros(N, nx)

200×597 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

In [15]:
ψbis

4×200 Array{Float64,2}:
 0.629605   0.326684  0.874646   0.0411056  …  0.363469  0.317491  0.934818
 0.22399    0.283091  0.53219    0.4247        0.527785  0.164593  0.871868
 0.566864   0.511157  0.700122   0.348717      0.723372  0.334182  0.952003
 0.0505716  0.995667  0.0391326  0.143356      0.769447  0.401063  0.300964

In [20]:
F.Q'*(F.Q*ψbis')

┌ Error: Failed to revise /media/mat/HDD/AdaptiveTransportMap/src/radialmap/greedyfit.jl
│   exception = (InterruptException(), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00007efdf81ec367, Ptr{Nothing} @0x00007efdf81f1bf6, Ptr{Nothing} @0x00007efdf816fc0e, Ptr{Nothing} @0x00007efdf8173597, Ptr{Nothing} @0x00007efdbcf54771])
└ @ Revise /home/mat/.julia/packages/Revise/AIcio/src/packagedef.jl:709
┌ Warning: The running code does not match the saved version for the following files:
│ 
│   /media/mat/HDD/AdaptiveTransportMap/src/radialmap/greedyfit.jl
│ 
│ If the error was due to evaluation order, it can sometimes be resolved by calling `Revise.retry()`.
│ Use Revise.errors() to report errors again. Only the first error in each file is shown.
│ Your prompt color may be yellow until the errors are resolved.
└ @ Revise /home/mat/.julia/packages/Revise/AIcio/src/packagedef.jl:805


200×4 Array{Float64,2}:
 0.629605   0.22399     0.566864  0.0505716
 0.326684   0.283091    0.511157  0.995667
 0.874646   0.53219     0.700122  0.0391326
 0.0411056  0.4247      0.348717  0.143356
 0.613903   0.632356    0.325746  0.514586
 0.902662   0.328683    0.525948  0.842128
 0.98988    0.169419    0.476712  0.805326
 0.540091   0.635358    0.622532  0.570431
 0.814644   0.160029    0.252947  0.960906
 0.718986   0.192323    0.140789  0.801022
 0.304391   0.534458    0.226681  0.085667
 0.948703   0.749022    0.898236  0.337926
 0.99419    0.911562    0.932376  0.711603
 ⋮                                
 0.194676   0.53485     0.209457  0.452462
 0.590701   0.250214    0.258469  0.0182
 0.0722457  0.983296    0.896947  0.303564
 0.406428   0.338016    0.288047  0.523898
 0.860231   0.941426    0.232003  0.297934
 0.0754768  0.00051333  0.485187  0.265898
 0.668773   0.158696    0.60858   0.0370232
 0.715556   0.586149    0.585833  0.582811
 0.77708    0.747355    0.471161  0.8

In [42]:
F = qrfactUnblocked(zeros(0,0))

QR{Float64,Array{Float64,2}}
Q factor:
0×0 LinearAlgebra.QRPackedQ{Float64,Array{Float64,2}}
R factor:
0×0 Array{Float64,2}

In [39]:
push!(tmp, zeros(5))

LoadError: [91mMethodError: [0mCannot `convert` an object of type [92mArray{Float64,1}[39m[0m to an object of type [91mFloat64[39m[39m
[91m[0mClosest candidates are:[39m
[91m[0m  convert(::Type{T}, [91m::Static.StaticInt{N}[39m) where {T<:Number, N} at /home/mat/.julia/packages/Static/BGJW6/src/static_implementation.jl:22[39m
[91m[0m  convert(::Type{T}, [91m::Static.StaticFloat64{N}[39m) where {N, T<:AbstractFloat} at /home/mat/.julia/packages/Static/BGJW6/src/float.jl:26[39m
[91m[0m  convert(::Type{T}, [91m::VectorizationBase.LazyMulAdd{M,O,I}[39m) where {M, O, I, T<:Number} at /home/mat/.julia/packages/VectorizationBase/4k8ZW/src/lazymul.jl:17[39m
[91m[0m  ...[39m

In [40]:
p = 2
order = p*ones(Int64, Nx)
order[1] = p
order[end] = p

order = [p; p; -1; p; p]

C = SparseRadialMapComponent(Nx, order)
center_std!(C, sort(X; dims = 2); γ = γ)

@time xdiag = optimize(C, X, λ, δ)
modify_a!(C, xdiag)

  0.000746 seconds (5.15 k allocations: 695.750 KiB)


In [42]:
    # Compute centers and widths
    center_std!(C, sort(X; dims = 2); γ = γ)

    ### Evaluate the different basis

    # Create weights
    ψ_off, ψ_diag, dψ_diag = compute_weights(C, X);

In [43]:
ψ_off

9×100 Array{Float64,2}:
 -1.8943      0.524572  -0.377613  -0.0761536  …   0.384642  -3.25978
  0.0948513   0.291842   0.433812   0.423307       0.33072    0.00231141
  0.0349627   0.402897   0.365975   0.420995       0.42299    0.0004043
  0.093796    0.158934   0.980648   1.46379       -0.853686   0.210463
  0.386148    0.375565   0.178475   0.0819955      0.344625   0.366209
  0.411406    0.413904   0.301441   0.178004   …   0.224426   0.414543
  0.384716   -0.228189   0.595606  -1.6628        -0.711736  -0.0878159
  0.404086    0.534755   0.31367    0.0726118      0.413267   0.532475
  0.519375    0.455307   0.46454    0.023578       0.254256   0.49821

In [56]:
@time F.Q*(F.Q'*ψ_diag')

  0.000079 seconds (10 allocations: 7.391 KiB)


100×4 Array{Float64,2}:
 -0.00707087  0.924713    0.738385     0.37348
 -5.24541e-7  0.999992    0.993738     1.30523
 -0.0608338   0.62681     0.467749     0.18046
 -0.0969348   0.505746    0.389569     0.143246
 -0.112763    0.462505    0.362976     0.131668
 -0.891696    0.00621773  0.0284838    0.0128753
 -0.0484145   0.678355    0.503694     0.199457
 -0.0676156   0.601149    0.450559     0.171827
 -0.367903    0.122654    0.149201     0.0524203
 -0.169119    0.340947    0.28966      0.102052
 -0.0627705   0.619319    0.462688     0.177889
 -0.107679    0.475863    0.371142     0.135171
 -0.157954    0.361737    0.302165     0.106887
  ⋮                                    
 -0.00738085  0.922004    0.734416     0.369397
 -0.088528    0.530854    0.405258     0.150316
 -0.0744019   0.576978    0.434716     0.164108
 -0.198409    0.292461    0.260307     0.0910113
 -0.149867    0.377701    0.311757     0.110653
 -0.0496635   0.672873    0.499767     0.197315
 -0.0213552   0.819936  

In [52]:
Matrix(F.Q')[1:Ne,:]*ψ_diag'

100×4 Array{Float64,2}:
  0.259283   -1.26526     -1.14093    -1.10693
  2.43358    -4.98023     -4.15672    -3.03175
 -0.113712    1.00018      0.839181    0.0215528
  0.238057    0.185024     0.180396    0.246032
 -0.527554   -1.92636     -1.58001    -1.23033
 -0.0281731  -0.576664    -0.454201   -0.117531
  0.0324086   0.258376     0.112765   -0.40009
  0.999858   -0.360184    -0.288807    0.0484478
  0.189334   -0.356912    -0.284213   -0.163268
  0.168473    0.0464709    0.0292691  -0.050503
  0.324687    0.0042863   -0.0607825  -0.230189
  0.0883382   0.0648405    0.0460566   0.0261449
  0.0130686  -0.0570624   -0.0624704  -0.312733
  ⋮                                   
  0.236156    0.155029     0.0823092  -0.185805
  0.060593   -0.00593668  -0.0453959  -0.131984
  0.132133    0.524442     0.388536    0.193398
  0.0585351  -0.425783    -0.352708   -0.397807
  0.109684   -0.0136371   -0.0583209  -0.322824
  0.190798    0.189964     0.0581992  -0.237499
  0.185859    0.0863229   

In [44]:
F = qr(ψ_off')

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
100×100 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.17235     -0.0123195   0.0195756   …   0.0552232   -0.342795
  0.0477271   -0.090474    0.140895       -0.131694     0.00507162
 -0.0343564   -0.124909   -0.0934567       0.143054     0.15335
 -0.00692869  -0.124303   -0.00655872      0.0112888   -0.0174998
  0.0110816   -0.116911    0.05368        -0.08452     -0.0441607
  0.039191    -0.0974732   0.127091    …   0.129849    -0.180872
 -0.00775866  -0.124519   -0.00939141     -0.131624     0.0209253
  0.181403    -0.0221591  -0.0599955       0.0374523    0.0550513
 -0.133454    -0.0445787  -0.0776178       0.0188524    0.047145
  0.0575559   -0.0822022   0.150916       -0.0249855   -0.0082464
 -0.00117364  -0.122488    0.0130785   …   0.00616488  -0.0233737
  0.13222     -0.0328142   0.0549326       0.0011328    0.00471768
 -0.0266938   -0.126127   -0.0713438      -0.0067487   -0.0146783
  ⋮                  

In [159]:
@testset "Verify updateqrfactUnblocked! for a vector" begin

    ψ = randn(100, 20)
    ϕ = randn(100)

    F = qrfactUnblocked(ψ)
    Fold = deepcopy(F)
    G = qrfactUnblocked(hcat(ψ, ϕ))
    F = updateqrfactUnblocked!(F, ϕ)
    @test norm(F.τ-G.τ)<1e-14
    @test norm(F.factors - G.factors)<1e-14
end

@testset "Verify updateqrfactUnblocked! for a matrix" begin

    ψ = randn(100, 20)
    ϕ = randn(100, 5)

    F = qrfactUnblocked(ψ)
    Fold = deepcopy(F)
    G = qrfactUnblocked(hcat(ψ, ϕ))
    F = updateqrfactUnblocked!(F, ϕ)
    @test norm(F.τ-G.τ)<1e-14
    @test norm(F.factors - G.factors)<1e-14
end

Test Summary:                              | Pass  Total
Verify updateqrfactUnblocked! for a vector |    2      2
Test Summary:                              | Pass  Total
Verify updateqrfactUnblocked! for a matrix |    2      2


Test.DefaultTestSet("Verify updateqrfactUnblocked! for a matrix", Any[], 2, false)